In [3]:
import torch

torch.set_printoptions(threshold=torch.inf)

# 加载序列化的 .pt 文件
file_path = '/data/processed/real3dad/train/807_sink_cut.pt'  # 替换为实际的文件路径

import torch, numpy as np, pprint

# 让 numpy / torch 打印不省略
np.set_printoptions(threshold=np.inf, linewidth=200, suppress=True)
try:
    torch.set_printoptions(profile="full")  # PyTorch 全量打印
except TypeError:
    torch.set_printoptions(threshold=float('inf'), linewidth=200)

def to_numpy(x):
    if isinstance(x, np.ndarray):
        return x
    if torch.is_tensor(x):
        return x.detach().cpu().numpy()
    return np.asarray(x)

def print_full(name, arr, as_int=False):
    a = to_numpy(arr)
    if as_int:
        a = a.astype(int)
    print(f"\n[{name}] shape={a.shape}, dtype={a.dtype}")
    # 用 array2string 强制完整展开
    print(np.array2string(a, max_line_width=200, separator=', '))

# 载入 .pt
data = torch.load(file_path, map_location='cpu')

# 元信息
print("\n[meta]")
pprint.pprint(data['meta'], width=120)

# 全量打印各字段
print_full('points', data['points'])                       # (N,3)
print_full('semantic_labels_pp', data['semantic_labels_pp'])   # (N,)
print_full('instance_labels_pp', data['instance_labels_pp'])   # (N,)
print_full('instance_masks', data['instance_masks'])           # (K,N) 布尔
print_full('semantic_labels_inst', data['semantic_labels_inst'])  # (K,)


[meta]
{'category': 'sink', 'category_id': 2, 'file': '/data/Real3D-AD-PCD/gemstone/test_neo/807_sink_cut.pcd'}

[points] shape=(2048, 3), dtype=float32
[[101.322914  ,   5.2357063 , 212.1126    ],
 [102.03042   ,   7.4388804 , 211.8871    ],
 [105.60603   ,   8.779505  , 211.8434    ],
 [108.52156   ,   9.452567  , 212.61201   ],
 [111.11454   ,   7.0797763 , 212.71826   ],
 [113.05215   ,   3.9314194 , 212.90398   ],
 [114.27      ,   1.2377977 , 212.13063   ],
 [114.57068   ,   3.764416  , 213.81187   ],
 [111.97815   ,   7.159434  , 213.26382   ],
 [110.76128   ,   9.434824  , 213.44406   ],
 [112.63795   ,   7.29658   , 213.94807   ],
 [114.101     ,   5.744659  , 214.5873    ],
 [114.948746  ,   4.1469417 , 214.38446   ],
 [116.06203   ,   2.6029994 , 214.35841   ],
 [114.60735   ,   0.7240813 , 211.99406   ],
 [116.36529   ,   1.6453097 , 214.57176   ],
 [115.54159   ,  -0.6808136 , 215.76492   ],
 [114.646164  ,  -3.060761  , 216.82397   ],
 [113.76351   ,  -5.325533  , 217.73

In [ ]:
# GT一致性检查
import numpy as np, torch
from pathlib import Path

root = Path("/data/processed/real3dad")
scene = "205_good_cut"

pt = torch.load(root/"val"/f"{scene}.pt", map_location="cpu")
N_pt = pt["points"].shape[0]

gt = np.loadtxt(root/"instance_gt"/"val"/f"{scene}.txt", dtype=np.int64)
print("N_pt =", N_pt, " N_gt =", gt.shape[0], " equal? ", N_pt == gt.shape[0])
print("unique semantic_id in GT:", np.unique(gt//1000))
print("unique inst_index in GT:", np.unique(gt%1000))

N_pt = 2048  N_gt = 2048  equal?  True
unique semantic_id in GT: [0]
unique inst_index in GT: [0]


In [5]:
import torch, glob, numpy as np

bad = []
empty = 0
for fp in glob.glob('/data/processed/real3dad/train/*.pt'):
    obj = torch.load(fp, map_location='cpu')
    K = obj['instance_masks'].shape[0]
    if K == 0:
        empty += 1
        continue
    sem_inst = np.asarray(obj['semantic_labels_inst'])
    if sem_inst.min() < 0 or sem_inst.max() > 1:
        bad.append(fp)

print("train empty samples:", empty)    # 期望 0（你 convert 里 keep_empty=False）
print("label out of {0,1} files:", len(bad))
if bad[:5]: print("examples:", bad[:5])

train empty samples: 0
label out of {0,1} files: 0
